In [16]:
import os
import pandas as pd
from sklearn.preprocessing import LabelBinarizer
import tensorflow as tf
from google.colab import drive
import glob
import keras
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [2]:
drive.mount('/content/drive/', force_remount=True)

Mounted at /content/drive/


In [3]:
filepath = glob.glob('/content/drive/My Drive/Programs/SB-FishDisease/SB-FishDisease/*/*.jpg')
labels = [x.split('/')[-2] for x in filepath]
labelsdf = pd.DataFrame(labels, columns=['Types'])

In [4]:
len(filepath)

349

In [5]:
# Initialize LabelBinarizer
lb = LabelBinarizer()

# Fit and transform the class labels
transformed_labels = lb.fit_transform(labelsdf['Types'])

# Adding One-Hot Encoded columns to the DataFrame with descriptive column names
for i, class_name in enumerate(lb.classes_):
    labelsdf[class_name] = transformed_labels[:, i]

# Optionally, if you want to see the DataFrame
print(labelsdf)

                                 Types  Bacterial Red disease  \
0    Viral diseases White tail disease                      0   
1    Viral diseases White tail disease                      0   
2    Viral diseases White tail disease                      0   
3    Viral diseases White tail disease                      0   
4    Viral diseases White tail disease                      0   
..                                 ...                    ...   
344  Bacterial diseases - Aeromoniasis                      0   
345  Bacterial diseases - Aeromoniasis                      0   
346  Bacterial diseases - Aeromoniasis                      0   
347  Bacterial diseases - Aeromoniasis                      0   
348  Bacterial diseases - Aeromoniasis                      0   

     Bacterial diseases - Aeromoniasis  Bacterial gill disease  \
0                                    0                       0   
1                                    0                       0   
2                    

In [6]:
# Define load_image function
def load_image(filepath):
    image = tf.io.read_file(filepath)
    image = tf.image.decode_jpeg(image, 3)
    image = tf.image.resize(image, [224, 224])
    image = tf.cast(image / 127.5 - 1, tf.float32)
    return image

In [7]:
# Create a Tensorflow dataset with only image and class label. NOTE: This will yield to an image-label pairing.
classification_dataset = tf.data.Dataset.from_tensor_slices((filepath, transformed_labels))
classification_dataset = classification_dataset.map(lambda img, lbl: (load_image(img), lbl))

# Batch constant
BATCH_SIZE = 16
AUTO = tf.data.experimental.AUTOTUNE

# Batch and prefetch
classification_dataset = classification_dataset.repeat().shuffle(500).batch(BATCH_SIZE)
classification_dataset = classification_dataset.prefetch(AUTO)

# Split dataset
test_count = int(len(filepath) * 0.2)
train_count = len(filepath) - test_count

train_dataset = classification_dataset.skip(test_count)
test_dataset = classification_dataset.take(test_count)

In [8]:
# Model architecture
conv = keras.applications.xception.Xception(weights='imagenet',
                                            include_top=False,
                                            input_shape=(224, 224, 3),
                                            pooling='avg')

# Define only the classification part of the model
inputs = keras.Input(shape=(224, 224, 3))
x = conv(inputs)
x = keras.layers.Dense(1024, activation='relu')(x)
x = keras.layers.Dropout(0.5)(x)
x = keras.layers.Dense(512, activation='relu')(x)
out_class = keras.layers.Dense(7, activation='softmax', name='out_item')(x)

# Create the model
classification_model = keras.models.Model(inputs=inputs, outputs=out_class)
classification_model.summary()

83683744/83683744 [==============================] - 5s 0us/step
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 xception (Functional)       (None, 2048)              20861480  
                                                                 
 dense (Dense)               (None, 1024)              2098176   
                                                                 
 dropout (Dropout)           (None, 1024)              0         
                                                                 
 dense_1 (Dense)             (None, 512)               524800    
                                                                 
 out_item (Dense)            (None, 7)                 3591      
                                                              

In [9]:
# Compile the model
classification_model.compile(optimizer=keras.optimizers.Adam(0.0003),
                             loss='categorical_crossentropy',
                             metrics=['acc'])

In [10]:
# Training
history = classification_model.fit(train_dataset,
                                   steps_per_epoch=train_count // BATCH_SIZE,
                                   epochs=50,
                                   validation_data=test_dataset,
                                   validation_steps=test_count // BATCH_SIZE)

Epoch 1/50
17/17 [==============================] - 71s 331ms/step - loss: 1.4980 - acc: 0.4007 - val_loss: 1.4971 - val_acc: 0.6250
Epoch 2/50
17/17 [==============================] - 4s 264ms/step - loss: 1.1322 - acc: 0.5588 - val_loss: 1.3590 - val_acc: 0.6562
Epoch 3/50
17/17 [==============================] - 4s 261ms/step - loss: 0.9759 - acc: 0.6618 - val_loss: 2.1963 - val_acc: 0.5938
Epoch 4/50
17/17 [==============================] - 4s 260ms/step - loss: 0.8168 - acc: 0.7574 - val_loss: 0.1843 - val_acc: 0.9219
Epoch 5/50
17/17 [==============================] - 4s 262ms/step - loss: 0.3801 - acc: 0.8713 - val_loss: 0.1762 - val_acc: 0.9219
Epoch 6/50
17/17 [==============================] - 4s 260ms/step - loss: 0.2663 - acc: 0.9338 - val_loss: 0.8253 - val_acc: 0.8438
Epoch 7/50
17/17 [==============================] - 4s 262ms/step - loss: 0.2980 - acc: 0.8934 - val_loss: 0.0465 - val_acc: 1.0000
Epoch 8/50
17/17 [==============================] - 4s 263ms/step - loss: 0

In [11]:
### VALIDATION PROCESS

In [46]:
# Function to load and preprocess a single image
def load_and_preprocess_single_image(image_path):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, 3)
    image = tf.image.resize(image, [224, 224])
    image = tf.cast(image / 127.5 - 1, tf.float32)
    image = tf.expand_dims(image, axis=0)  # Expand dimensions to fit the model's input shape (1, 224, 224, 3)
    return image

# Provide the path to the image you want to validate
validation_image_path = '/content/drive/My Drive/photos/lice_anchor-worms_gill-maggots_1.jpg'

# Load and preprocess the validation image
validation_image = load_and_preprocess_single_image(validation_image_path)

# Get predictions from the trained model
predictions = classification_model.predict(validation_image)

# Decode the predictions to class labels and print the predicted label
predicted_label = lb.classes_[np.argmax(predictions)]
print(f"The predicted label for the uploaded image is: {predicted_label}")

1/1 [==============================] - 0s 23ms/step
The predicted label for the uploaded image is: Parasitic diseases
